# Sistema de Recomendación Basado en Palabras Clave

Este sistema de recomendación utiliza una selección de palabras clave para identificar similitudes entre películas y generar recomendaciones. En lugar de analizar todo el contenido de las películas, el sistema se basa únicamente en una lista específica de palabras clave para optimizar el rendimiento.

## Palabras Clave Seleccionadas

Las palabras clave seleccionadas representan temas y conceptos comunes en títulos de películas que podrían ser útiles para el sistema de recomendación. Estas palabras clave son:

- **Night**, **Day**, **Love**, **Man**, **Girl**, **Last**, **Movie**, **Time**, **World**

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json

# Cargar los datos
movies_df = pd.read_csv('final_data/optimized_movies.csv')

# Lista de palabras clave
keywords = ["Night", "Day", "Love", "Man", "Girl", "Last", "Movie", "Time", "World"]

# Función para filtrar solo las palabras clave en una cadena de texto
def filter_keywords(text, keywords):
    words = text.split()
    return ' '.join([word for word in words if word in keywords])

# Crear la columna 'content' con palabras clave en título, géneros y descripción
movies_df['content'] = (
    movies_df['title'].fillna('') + " " + 
    movies_df['genres_names'].fillna('') + " " + 
    movies_df['overview'].fillna('')
).apply(lambda x: filter_keywords(x, keywords))

# Rellenar valores NaN en la columna 'content' con cadenas vacías
movies_df['content'] = movies_df['content'].fillna('')

# Vectorización TF-IDF basada en las palabras clave
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(movies_df['content'])

# Cálculo de la similitud de coseno
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Crear recomendaciones
recommendations = {}
for idx, title in enumerate(movies_df['title']):
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:6]
    movie_indices = [i[0] for i in sim_scores]
    recommendations[title] = movies_df['title'].iloc[movie_indices].tolist()

# Guardar recomendaciones en un archivo JSON en la carpeta final_data
with open('final_data/recommendations.json', 'w') as f:
    json.dump(recommendations, f)

In [10]:
import json

# Cargar el archivo completo de recomendaciones
with open('final_data/recommendations.json', 'r') as f:
    recommendations = json.load(f)

# Limitar a solo las primeras 100 películas
reduced_recommendations = dict(list(recommendations.items())[:100])

# Guardar el archivo reducido
with open('final_data/recommendations.json', 'w') as f:
    json.dump(reduced_recommendations, f)

print("Archivo recommendations.json reducido a las primeras 100 películas.")

Archivo recommendations.json reducido a las primeras 100 películas.


In [11]:
import json
import csv

# Cargar el archivo JSON de recomendaciones
with open('final_data/recommendations.json', 'r') as f:
    recommendations = json.load(f)

# Convertir a CSV
with open('final_data/recommendations.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    # Escribir la cabecera
    writer.writerow(["title", "recommendation_1", "recommendation_2", "recommendation_3", "recommendation_4", "recommendation_5"])
    
    # Escribir cada película y sus recomendaciones en una fila
    for title, recs in recommendations.items():
        # Asegurarse de que cada fila tiene exactamente 5 recomendaciones (rellenar con None si tiene menos)
        row = [title] + recs[:5] + [None] * (5 - len(recs))
        writer.writerow(row)

print("Archivo recommendations.csv creado exitosamente.")

Archivo recommendations.csv creado exitosamente.


## Ventajas de Este Enfoque

- **Optimización de Rendimiento**: Al limitar el análisis a palabras clave específicas, se reduce significativamente la carga de procesamiento y se mejora el tiempo de respuesta.
- **Recomendaciones Relevantes**: Las palabras clave seleccionadas representan conceptos comunes en títulos de películas, lo que permite obtener recomendaciones que podrían ser temáticamente similares.
- **Uso de Recursos Reducido**: Guardar las recomendaciones en un archivo JSON permite evitar cálculos intensivos de similitud en cada solicitud, haciendo el sistema más adecuado para entornos con recursos limitados.

Este sistema de recomendación optimizado es ideal para servicios en la nube con restricciones de memoria y potencia de procesamiento, proporcionando recomendaciones relevantes de manera eficiente.